#### Importing all Relevant Libraries

In [0]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

#### 1. [10 pts] What was the average time each driver spent at the pit stop for each race?

In [0]:
# Importing relevant data and converting to pandas dataframe 
# I find it easier to deal with pandas instead of spark
df_pit_stops = spark.read.csv('s3://columbia-gr5069-main/raw/pit_stops.csv', header=True)
df_pit_stops = df_pit_stops.toPandas()
df_drivers = spark.read.csv('s3://columbia-gr5069-main/raw/drivers.csv', header=True)
df_drivers = df_drivers.toPandas()
df_races = spark.read.csv('s3://columbia-gr5069-main/raw/races.csv', header=True)
df_races = df_races.toPandas()  


In [0]:
# Quick overview of pitstop dataframe
df_pit_stops.head()

In [0]:
#Running a group by function to get each driver's avg time (in miliseconds) at pit stop at each race
avg_pit_stop_time = pd.DataFrame(df_pit_stops.groupby(['raceId', 'driverId'])['milliseconds'].mean()).reset_index()

In [0]:
avg_pit_stop_time.head()

In [0]:
# Overview of driver information
df_drivers.head()

In [0]:
#Overview of Races
df_races.head()

In [0]:
#Matching on driver ID to get name of drivers on pit_stops
avg_pit_stop_time = pd.merge(avg_pit_stop_time, df_drivers[['driverId', 'forename', 'surname']], on='driverId', how='left')
#I add driver ID to Name, just to make it easier for me too visualize / see. no need to necessarily add it
avg_pit_stop_time['Name + id'] = avg_pit_stop_time['forename'] + ' ' +avg_pit_stop_time['surname'] + ' ' +avg_pit_stop_time ['driverId']
avg_pit_stop_time.drop(['forename', 'surname'], axis=1, inplace=True)
avg_pit_stop_time.head()

In [0]:
# merging on RaceId to get each unique race 
# I use race name and date of race to make sure I am getting all races
avg_pit_stop_time = pd.merge(avg_pit_stop_time, df_races[['raceId', 'name', 'date']], on='raceId', how='left')
avg_pit_stop_time['Race Name + date'] = avg_pit_stop_time['name'] + ' ' + avg_pit_stop_time['date']
avg_pit_stop_time.drop(['name','date'], axis=1, inplace=True)
avg_pit_stop_time.head()

#### 2. [20 pts] Rank the average time spent at the pit stop in order of who won each race


In [0]:
#I am getting the results dataset. 
df_results = spark.read.csv('s3://columbia-gr5069-main/raw/results.csv', header=True)
df_results = df_results.toPandas()

In [0]:
#Quick overview of player results per race
df_results.head()

In [0]:
#Merging on raceId and driver Id to get points for each player
# I use points because that way i can get all drivers, even those that did not finish the race and those that have 0 points
# I only deal with the ones not finishing the race as part of Q5.
avg_pit_stop_time = pd.merge(avg_pit_stop_time, df_results[['raceId', 'driverId', 'points']], on=['raceId', 'driverId'], how='left')
avg_pit_stop_time.head()

In [0]:
#Making sure raceId and points is categorized as int so that I can sort the values by raceiD and then points
avg_pit_stop_time.raceId = avg_pit_stop_time.raceId.astype(int)
avg_pit_stop_time.points = avg_pit_stop_time.points.astype(int)
avg_pit_stop_time = avg_pit_stop_time.sort_values(by=['raceId', 'points'], ascending=False)

In [0]:
#Now, for each race, players are ranked based on highest points to lowest points
avg_pit_stop_time

#### 3. [20 pts] Insert the missing code (e.g: ALO for Alonso) for drivers based on the 'drivers' dataset


In [0]:
# No codes are missing
df_drivers['code'].isnull().sum()

In [0]:
#Matching on driver ID to get code from drivers dataset
avg_pit_stop_time = pd.merge(avg_pit_stop_time, df_drivers[['driverId', 'code']], on='driverId', how='left')
avg_pit_stop_time.head()

#### 4. [20 pts] Who is the youngest and oldest driver for each race? Create a new column called “Age”

In [0]:
#getting the dob from drivers dataset by merging on driverID
avg_pit_stop_time = pd.merge(avg_pit_stop_time, df_drivers[['driverId', 'dob']], on='driverId', how='left')

In [0]:
#I am stripping the date (aka last 10 characters) from 'Race Name + date' column to ge the data only. 
# I then save it as a column as a datetime object
avg_pit_stop_time['race_date'] = pd.to_datetime(avg_pit_stop_time['Race Name + date'].str[-10:])
avg_pit_stop_time.head()

In [0]:
#I am calculating the age of each player at the time of the race, by subtraction of dob year from race year
#I store results in 'Age at Race', then I drop the dob and race_date columns
avg_pit_stop_time['Age at Race'] =  avg_pit_stop_time['race_date'].dt.year - pd.to_datetime(avg_pit_stop_time['dob']).dt.year 
avg_pit_stop_time.drop(['dob', 'race_date'], axis=1, inplace=True)
avg_pit_stop_time.head()


In [0]:
#I find all the unique races and save it in an empty list
#I then create an dict with only keys, values, which are in the form of list, to be populated
all_unique_races = avg_pit_stop_time['Race Name + date'].unique()
youngest_oldest_per_race = {'Race Name': [], 'Youngest Player Name':[],'Youngest Player Age':[], 'Oldest Player Name':[], 'Oldest Player Age': []}

In [0]:
#This loop goes through all values in the all_unique_races list
for race in all_unique_races:
    # Appends each race to dictionary
    youngest_oldest_per_race['Race Name'].append(race)
    # Subset dataset to only look at the race we are looping on
    unique_race_subset = avg_pit_stop_time[avg_pit_stop_time['Race Name + date']==race]
    # Sort the new subset dataset by age
    unique_race_subset = unique_race_subset.sort_values(by=['Age at Race'])
    #Now, we know that first row has youngest player details and last row has oldest player details
    # We use this to populate the empty lists in the earlier dictionaries
    youngest_oldest_per_race['Youngest Player Name'].append(unique_race_subset.iloc[0]['Name + id'])
    youngest_oldest_per_race['Youngest Player Age'].append(unique_race_subset.iloc[0]['Age at Race'])
    youngest_oldest_per_race['Oldest Player Name'].append(unique_race_subset.iloc[-1]['Name + id'])
    youngest_oldest_per_race['Oldest Player Age'].append(unique_race_subset.iloc[-1]['Age at Race'])


In [0]:
#Converting dictionary to DF for easier interpretability

youngest_oldest_per_race = pd.DataFrame(youngest_oldest_per_race)
youngest_oldest_per_race.head()

#### 5. [20 pts] For a given race, which driver has the most wins and losses?

In [0]:
avg_pit_stop_time.head()

In [0]:
# I create new columns to get # of previous races won or not one (but completed)
avg_pit_stop_time['# of Previous Races Won (Excluding Current One)'] = [0] * len(avg_pit_stop_time)
avg_pit_stop_time['# of Previous Not Won, But Completed Races(Excluding Current One)'] = [0] * len(avg_pit_stop_time)

avg_pit_stop_time

In [0]:
# I turn raceId and driverID columns of dataframe I am going to use to type int to make the comparisons easier
avg_pit_stop_time['driverId'] = avg_pit_stop_time['driverId'].astype(int)
df_results['raceId'] = df_results['raceId'].astype(int)
df_results['driverId'] = df_results['driverId'].astype(int)

In [0]:
#I loop through each row in avg_pit_stop_time
for index, row in avg_pit_stop_time.iterrows():
    #I store the raceId and driverId
    raceId = row['raceId']
    driverId = row['driverId']
    #I then subset the results dataset to only get raceIds less than the current raceId for the specific driver
    # I do this b/c i know the lower raceId, means it happened before and we want to know about previous races excluding current one
    subset = df_results[(df_results['raceId'] < raceId) & (df_results['driverId'] == driverId)]
    #I then get a count of the subset data where position is 1, meaning they wone
    won_races = len(subset[subset['position']=='1'])
    # I also get a count where position is not 1 and time is not equal to \\N(meaning not completed)
    not_won_races = len(subset[(subset['position'] != '1') & (subset['time'] != '\\N')])
    #I then put these results in the new columns created for avg_pit_stop_time
    avg_pit_stop_time.loc[index, '# of Previous Races Won (Excluding Current One)'] = won_races
    avg_pit_stop_time.loc[index, '# of Previous Not Won, But Completed Races(Excluding Current One)'] = not_won_races


avg_pit_stop_time.head()

#### 6. [10 pts] How has the nationalities of drivers participating in F1 races changed since the year 2000? Create a dataframe showing count of each nationality by year. 

In [0]:
#I first merge df_results with df_races on raceId for to get the year of each race.
#I decided to use df_results since it has both raceId and driver ID, df_results only has raceId, no driverId
df_races['raceId'] = df_races['raceId'].astype(int)
nationalities = pd.merge(df_results, df_races[['raceId', 'year']], on='raceId', how='left')
nationalities.head()

In [0]:
#For my purposes, i simplify the dataset by only keeping raceId, driverId, and year
# I change year to an int to make slicing easier (2000 and above)
nationalities = nationalities[['raceId','driverId', 'year']]
nationalities['year'] = nationalities['year'].astype(int)
nationalities = nationalities[nationalities['year']>=2000]
nationalities

In [0]:
#I then short the values by year, just to make sure they are in order
nationalities.sort_values(by=['year'], inplace=True)

In [0]:
#I then merge data with drivers dataset to get nationality and forename(not necessary, but I like to have it) of each driver
df_drivers['driverId'] = df_drivers['driverId'].astype(int)
nationalities = pd.merge(nationalities, df_drivers[['driverId', 'forename', 'nationality']], on='driverId', how='left')
nationalities

In [0]:
#Right now, there are multiple duplicate rows. Essentially, for each year, the same player might show up 3 times because they played in multiple races
#I first drop raceId, since we only needed it to get the year
# I then drop all duplicates, ensuring that one driver is only counted once per year (dropping all its duplicates in the same year)
nationalities = nationalities[['driverId', 'year', 'forename', 'nationality']]
nationalities = nationalities.drop_duplicates()
nationalities

In [0]:
#Then, I create an empty dataframe
change_in_nationalities = pd.DataFrame()

In [0]:
# Right now, I have a dataframe with one row per each participating driver for each year and their associated nationality. I want one row per year, with count of each nationality
for year in nationalities['year'].unique():
    #So i loop through all unique years in the dataframe and slice the dataframe to only include the year I am looping on
    #I then run value counts on 'nationality' to get a count of the nationality. i convert the value count to a dataframe
    #I transpose the dataframe so that each nationality is a column instead of a row
    nationalities_in_year = pd.DataFrame(nationalities[nationalities['year']==year]['nationality'].value_counts()).transpose()
    #I then add the year I am looping on to the dataframe and reset the index
    nationalities_in_year.insert(loc = 0, column = 'year', value = year)    
    nationalities_in_year.reset_index(drop=True, inplace=True)
    #Finally I concatenate the new dataframe with the previous one empty one I created. This ensures that the result from each year I am looping on is added to the empty df
    change_in_nationalities = pd.concat([change_in_nationalities, nationalities_in_year], axis=0)

In [0]:
change_in_nationalities

In [0]:
# I can see that there are some NaNs. I convert them to 0s for better interpretability
#I know that fillna returns float, so I turn entire df astype int
change_in_nationalities = change_in_nationalities.fillna(0)
change_in_nationalities = change_in_nationalities.astype(int)
change_in_nationalities